# Problem 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the random seed for reproducibility
np.random.seed(42)

# Parameters
lambda_arrival = 2  # Arrival rate (customers per minute)
mu_service = 1      # Service rate (customers per minute)
num_customers = 10  # Number of customers to simulate
num_replications = 100  # Number of simulation replications

In [6]:
# Part (a): Naive Monte Carlo Simulation

total_time_in_system_list = []

for replication in range(num_replications):
    arrival_times = np.cumsum(np.random.exponential(scale=1/lambda_arrival, size=num_customers))
    service_times = np.random.exponential(scale=1/mu_service, size=num_customers)
    start_service_times = np.zeros(num_customers)
    completion_times = np.zeros(num_customers)
    time_in_system = np.zeros(num_customers)

    start_service_times[0] = arrival_times[0]
    completion_times[0] = start_service_times[0] + service_times[0]
    time_in_system[0] = completion_times[0] - arrival_times[0]

    for i in range(1, num_customers):
        start_service_times[i] = max(arrival_times[i], completion_times[i-1])
        completion_times[i] = start_service_times[i] + service_times[i]
        time_in_system[i] = completion_times[i] - arrival_times[i]

    total_time_in_system = np.sum(time_in_system)
    total_time_in_system_list.append(total_time_in_system)

point_estimate_naive = np.mean(total_time_in_system_list)
variance_per_replication_naive = np.var(total_time_in_system_list, ddof=1)

print("Part (a): Naive Monte Carlo Simulation")
print(f"Point Estimate: {point_estimate_naive:.4f} minutes")
print(f"Variance per Replication: {variance_per_replication_naive:.4f}")

Part (a): Naive Monte Carlo Simulation
Point Estimate: 38.2929 minutes
Variance per Replication: 457.9150


In [7]:
# Part (b): Control Variate with Total Service Time

expected_total_service_time = num_customers * (1 / mu_service)

total_time_in_system_list = []
total_service_time_list = []

for replication in range(num_replications):
    arrival_times = np.cumsum(np.random.exponential(scale=1/lambda_arrival, size=num_customers))
    service_times = np.random.exponential(scale=1/mu_service, size=num_customers)
    start_service_times = np.zeros(num_customers)
    completion_times = np.zeros(num_customers)
    time_in_system = np.zeros(num_customers)

    start_service_times[0] = arrival_times[0]
    completion_times[0] = start_service_times[0] + service_times[0]
    time_in_system[0] = completion_times[0] - arrival_times[0]

    for i in range(1, num_customers):
        start_service_times[i] = max(arrival_times[i], completion_times[i-1])
        completion_times[i] = start_service_times[i] + service_times[i]
        time_in_system[i] = completion_times[i] - arrival_times[i]

    total_time_in_system = np.sum(time_in_system)
    total_service_time = np.sum(service_times)

    total_time_in_system_list.append(total_time_in_system)
    total_service_time_list.append(total_service_time)

total_time_in_system_array = np.array(total_time_in_system_list)
total_service_time_array = np.array(total_service_time_list)

covariance = np.cov(total_time_in_system_array, total_service_time_array, ddof=1)[0, 1]
variance_service_time = np.var(total_service_time_array, ddof=1)
beta_optimal = covariance / variance_service_time

adjusted_estimates = total_time_in_system_array - beta_optimal * (total_service_time_array - expected_total_service_time)

point_estimate_cv = np.mean(adjusted_estimates)
variance_per_replication_cv = np.var(adjusted_estimates, ddof=1)

print("\nPart (b): Control Variate with Total Service Time")
print(f"Optimal Beta: {beta_optimal:.4f}")
print(f"Point Estimate: {point_estimate_cv:.4f} minutes")
print(f"Variance per Replication: {variance_per_replication_cv:.4f}")

variance_reduction = ((variance_per_replication_naive - variance_per_replication_cv) / variance_per_replication_naive) * 100
print(f"Variance Reduction: {variance_reduction:.2f}%")


Part (b): Control Variate with Total Service Time
Optimal Beta: 5.2783
Point Estimate: 36.2446 minutes
Variance per Replication: 91.9840
Variance Reduction: 79.91%


In [8]:
# Part (c): Control Variate with Total Service Time minus Total Interarrival Time

expected_total_service_time = num_customers * (1 / mu_service)
expected_total_interarrival_time = num_customers * (1 / lambda_arrival)
expected_control_variate = expected_total_service_time - expected_total_interarrival_time

total_time_in_system_list = []
control_variate_list = []

for replication in range(num_replications):
    interarrival_times = np.random.exponential(scale=1/lambda_arrival, size=num_customers)
    arrival_times = np.cumsum(interarrival_times)
    service_times = np.random.exponential(scale=1/mu_service, size=num_customers)
    start_service_times = np.zeros(num_customers)
    completion_times = np.zeros(num_customers)
    time_in_system = np.zeros(num_customers)

    start_service_times[0] = arrival_times[0]
    completion_times[0] = start_service_times[0] + service_times[0]
    time_in_system[0] = completion_times[0] - arrival_times[0]

    for i in range(1, num_customers):
        start_service_times[i] = max(arrival_times[i], completion_times[i-1])
        completion_times[i] = start_service_times[i] + service_times[i]
        time_in_system[i] = completion_times[i] - arrival_times[i]

    total_time_in_system = np.sum(time_in_system)
    total_service_time = np.sum(service_times)
    total_interarrival_time = np.sum(interarrival_times)
    control_variate = total_service_time - total_interarrival_time

    total_time_in_system_list.append(total_time_in_system)
    control_variate_list.append(control_variate)

total_time_in_system_array = np.array(total_time_in_system_list)
control_variate_array = np.array(control_variate_list)

covariance = np.cov(total_time_in_system_array, control_variate_array, ddof=1)[0, 1]
variance_control_variate = np.var(control_variate_array, ddof=1)
beta_optimal = covariance / variance_control_variate

adjusted_estimates = total_time_in_system_array - beta_optimal * (control_variate_array - expected_control_variate)

point_estimate_cv = np.mean(adjusted_estimates)
variance_per_replication_cv = np.var(adjusted_estimates, ddof=1)

print("\nPart (c): Control Variate with Total Service Time minus Total Interarrival Time")
print(f"Optimal Beta: {beta_optimal:.4f}")
print(f"Point Estimate: {point_estimate_cv:.4f} minutes")
print(f"Variance per Replication: {variance_per_replication_cv:.4f}")

variance_reduction = ((variance_per_replication_naive - variance_per_replication_cv) / variance_per_replication_naive) * 100
print(f"Variance Reduction compared to Naive Simulation: {variance_reduction:.2f}%")

variance_reduction_b = ((variance_per_replication_cv - variance_per_replication_cv) / variance_per_replication_cv) * 100
print(f"Variance Reduction compared to Part (b): {variance_reduction_b:.2f}%")


Part (c): Control Variate with Total Service Time minus Total Interarrival Time
Optimal Beta: 4.4266
Point Estimate: 34.7598 minutes
Variance per Replication: 81.8711
Variance Reduction compared to Naive Simulation: 82.12%
Variance Reduction compared to Part (b): 0.00%


# Problem 4

In [9]:
import numpy as np
from scipy.stats import beta, norm, bernoulli

# Set the random seed for reproducibility
np.random.seed(42)

# Parameters
N = 100  # Number of loans
alpha = 1  # Beta distribution parameter for P
beta_param = 19  # Beta distribution parameter for P
expected_P = alpha / (alpha + beta_param)  # Expected value of P
expected_Xn = 3  # Expected loss given default
variance_Xn = 1  # Variance of loss given default
x = 45  # Threshold for total loss
num_replications = 100  # Number of simulation replications

In [10]:
# Naive Monte Carlo Simulation

indicator_values = []

for replication in range(num_replications):
    P = np.random.beta(alpha, beta_param)

    D_n = np.random.binomial(1, P, size=N) 
    
    num_defaults = np.sum(D_n)
    X_defaults = np.random.normal(loc=3, scale=1, size=num_defaults)
    
    X_n = np.zeros(N)
    X_n[D_n == 1] = X_defaults
    
    L = np.sum(X_n)
    
    indicator = 1 if L > x else 0
    indicator_values.append(indicator)

point_estimate_naive = np.mean(indicator_values)

variance_naive = point_estimate_naive * (1 - point_estimate_naive)

print("Naive Monte Carlo Simulation")
print(f"Point Estimate of P(L > {x}): {point_estimate_naive:.4f}")
print(f"Variance per Replication: {variance_naive:.6f}")

Naive Monte Carlo Simulation
Point Estimate of P(L > 45): 0.0300
Variance per Replication: 0.029100


In [11]:
# Conditional Monte Carlo Simulation

PL_given_P_values = []

for replication in range(num_replications):
    P = np.random.beta(alpha, beta_param)
    
    mu_L_given_P = 3 * N * P  
    
    var_L_given_P = N * P * (1 + 9 * (1 - P))
    
    sigma_L_given_P = np.sqrt(var_L_given_P)
    
    if sigma_L_given_P > 0:
        z = (x - mu_L_given_P) / sigma_L_given_P
        PL_given_P = 1 - norm.cdf(z)
    else:
        PL_given_P = 1.0 if mu_L_given_P > x else 0.0
    
    PL_given_P_values.append(PL_given_P)

point_estimate_conditional = np.mean(PL_given_P_values)

variance_conditional = np.var(PL_given_P_values, ddof=1)

print("\nConditional Monte Carlo Simulation")
print(f"Point Estimate of P(L > {x}): {point_estimate_conditional:.4f}")
print(f"Variance per Replication: {variance_conditional:.6f}")


Conditional Monte Carlo Simulation
Point Estimate of P(L > 45): 0.0436
Variance per Replication: 0.016077


In [12]:
# Variance Reduction
variance_reduction = ((variance_naive - variance_conditional) / variance_naive) * 100

print("\nPerformance Comparison")
print(f"Variance Reduction: {variance_reduction:.2f}%")


Performance Comparison
Variance Reduction: 44.75%
